In [40]:
from torch.utils.data import Dataset, DataLoader# For custom data-sets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import torch
import pandas as pd
from collections import namedtuple

n_class = 27

# a label and all meta information
Label = namedtuple( 'Label' , [
    'name'        , 
    'level3Id'    , 
    'color'       , 
    ] )

labels = [
    #       name                     level3Id  color
    Label(  'road'                 ,    0  , (128, 64,128)  ),
    Label(  'drivable fallback'    ,    1  , ( 81,  0, 81)  ),
    Label(  'sidewalk'             ,    2  , (244, 35,232)  ),
    Label(  'non-drivable fallback',    3  , (152,251,152)  ),
    Label(  'person/animal'        ,    4  , (220, 20, 60)  ),
    Label(  'rider'                ,    5  , (255,  0,  0)  ),
    Label(  'motorcycle'           ,    6  , (  0,  0,230)  ),
    Label(  'bicycle'              ,   7  , (119, 11, 32)  ),
    Label(  'autorickshaw'         ,   8  , (255, 204, 54) ),
    Label(  'car'                  ,   9  , (  0,  0,142)  ),
    Label(  'truck'                ,  10 ,  (  0,  0, 70)  ),
    Label(  'bus'                  ,  11 ,  (  0, 60,100)  ),
    Label(  'vehicle fallback'     ,  12 ,  (136, 143, 153)),  
    Label(  'curb'                 ,   13 ,  (220, 190, 40)),
    Label(  'wall'                 ,  14 ,  (102,102,156)  ),
    Label(  'fence'                ,  15 ,  (190,153,153)  ),
    Label(  'guard rail'           ,  16 ,  (180,165,180)  ),
    Label(  'billboard'            ,   17 ,  (174, 64, 67) ),
    Label(  'traffic sign'         ,  18 ,  (220,220,  0)  ),
    Label(  'traffic light'        ,  19 ,  (250,170, 30)  ),
    Label(  'pole'                 ,  20 ,  (153,153,153)  ),
    Label(  'obs-str-bar-fallback' , 21 ,  (169, 187, 214) ),  
    Label(  'building'             ,  22 ,  ( 70, 70, 70)  ),
    Label(  'bridge/tunnel'        ,  23 ,  (150,100,100)  ),
    Label(  'vegetation'           ,  24 ,  (107,142, 35)  ),
    Label(  'sky'                  ,  25 ,  ( 70,130,180)  ),
    Label(  'unlabeled'            ,  26 ,  (  0,  0,  0)  ),
]   

class IddDataset(Dataset):

    def __init__(self, csv_file, n_class=n_class, transforms_=None):
        self.data      = pd.read_csv(csv_file)
        self.n_class   = n_class
        self.mode = csv_file
        
        # Add any transformations here
        
        # The following transformation normalizes each channel using the mean and std provided
        self.transforms = transforms.Compose([transforms.ToTensor(),
                                              transforms.CenterCrop((720, 1280)),
                                              transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        img_name = self.data.iloc[idx, 0]
        img = Image.open(img_name).convert('RGB')
        label_name = self.data.iloc[idx, 1]
        label = Image.open(label_name)
        
        img = np.asarray(img) / 255. # scaling [0-255] values to [0-1]
        label = np.asarray(label)
        
        img = self.transforms(img).float() # Normalization
        label = torch.from_numpy(label.copy()).long() # convert to tensor

        # create one-hot encoding
        h, w = label.shape
        target = torch.zeros(self.n_class, h, w)
        for c in range(self.n_class):
            target[c][label == c] = 1
        
        return img, target, label

In [41]:
train_dataset = IddDataset(csv_file='train.csv')
val_dataset = IddDataset(csv_file='val.csv')
test_dataset = IddDataset(csv_file='test.csv')


train_loader = DataLoader(dataset=train_dataset, batch_size= 128, num_workers=0, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 128, num_workers=0, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size= 128, num_workers=0, shuffle=False)

In [42]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

4018
2009
1004


In [43]:
import matplotlib.pyplot as plt

# Print the shape of the training dataset
# print(train_dataset[0][0].shape)
# Permute the object into a different shape
train_image = getitem(train_dataset, 0)
#plt.imshow(tensor_image.permute(1, 2, 0))
#plt.imshow(test_image.permute(1, 2, 0))
#plt.imshow(val_image.permute(1, 2, 0))

NameError: name 'getitem' is not defined

In [ ]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

# First read the dataset
train_dataset = IddDataset(csv_file='train.csv')
val_dataset = IddDataset(csv_file='val.csv')
test_dataset = IddDataset(csv_file='test.csv')

# train_loader, val_loader and test_loader are three different sets of data
train_loader = DataLoader(dataset=train_dataset, batch_size= 128, num_workers=0, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 128, num_workers=0, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size= 128, num_workers=0, shuffle=False)

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        print("Dimension of weight = {}".format(m.weight.shape))
        print("Dimension of bias = {}".format(m.bias.shape))
        #torch.nn.init.xavier_uniform(m.bias.data)
        torch.nn.init.zeros_(m.bias)
        
epochs = 100        
criterion = torch.nn.BCELoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)

optimizer = optim.Adam(fcn_model.parameters(), lr=10**-4)

use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()

        
def train():
    print("Reached train function")
    for epoch in range(epochs):
        ts = time.time()
        print("Epoch: {}".format(epoch))
        for itera, (X, tar, Y) in enumerate(train_loader):
            print("Printing the contents of X, tar and Y")
            print("X, tar, Y are {}, {} and {}".format(type(X), type(tar), type(Y)))
            optimizer.zero_grad()
            if use_gpu:
                inputs = X.cuda()# Move your inputs onto the gpu
                labels = Y.cuda()# Move your labels onto the gpu
            else:
                inputs, labels = X, Y# Unpack variables into inputs and labels

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, itera, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val(epoch)
        fcn_model.train()
    


def val(epoch):
    fcn_model.eval() # Don't forget to put in eval mode !
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    
def test():
    fcn_model.eval()
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
if __name__ == "__main__":
    #val(0)  # show the accuracy before training
    train()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Dimension of weight = torch.Size([32, 3, 3, 3])
Dimension of bias = torch.Size([32])
Dimension of weight = torch.Size([64, 32, 3, 3])
Dimension of bias = torch.Size([64])
Dimension of weight = torch.Size([128, 64, 3, 3])
Dimension of bias = torch.Size([128])
Dimension of weight = torch.Size([256, 128, 3, 3])
Dimension of bias = torch.Size([256])
Dimension of weight = torch.Size([512, 256, 3, 3])
Dimension of bias = torch.Size([512])
Dimension of weight = torch.Size([512, 512, 3, 3])
Dimension of bias = torch.Size([512])
Dimension of weight = torch.Size([512, 256, 3, 3])
Dimension of bias = torch.Size([256])
Dimension of weight = torch.Size([256, 128, 3, 3])
Dimension of bias = torch.Size([128])
Dimension of weight = torch.Size([128, 64, 3, 3])
Dimension of bias = torch.Size([64])
Dimension of weight = torch.Size([64, 32, 3, 3])
Dimension of bias = torch.Size([32])
Dimension of weight = torch.Size([27, 32, 1, 1])
Dimension of bias = torch.Size([27])
Reached train function
Epoch: 0
